In [1]:
import pixiedust
#!pip install --upgrade ~/Development/repos/quantutils
#!pip install --upgrade ~/Development/repos/tradeframework


# import utilities
from quantutils.api.datasource import MarketDataStore
from quantutils.api.auth import CredentialsStore
import quantutils.dataset.pipeline as ppl

#import tradeframework
from tradeframework.api import Derivative, Optimizer, Model, TradeEnvironment, TradeEngine, Asset
from tradeframework.environments import SandboxEnvironment, ZiplineEnvironment
from tradeframework.models import PreOpenMomentum
import tradeframework.operations.trader as trader
import tradeframework.operations.utils as utils

# Development
import pandas
import numpy as np


%matplotlib inline

Pixiedust database opened successfully


/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'
/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
## Get Market Data

mds = MarketDataStore("../datasources")

assets = {
    "markets": ["DOW"],
    "start": "2013-01-01",
    "end": "2013-07-11"
}

#assets = {
#    "markets": ["DOW"],
#    "start": "2013-01-01",
#    "end": "2018-08-01"
#}

markets = mds.loadMarketData(assets, "H")
ts = ppl.removeNaNs(markets["DOW"])

sample1 = ts[5:10].copy()
sample2 = ts[12:15].copy()
sample3 = ts[:5].copy()

/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Loading DOW data from WallSt-hourly in tradefair.hdf
Resampling to H periods
Resampling to H periods
Merging data...
Loading DOW data from D&J-IND in finam.hdf
Resampling to H periods
Resampling to H periods
Merging data...


In [6]:
sample1

,Open,High,Low,Close
Date_Time,,,,
2013-01-02 20:00:00+00:00,13331.83,13395.20,13331.83,13395.20
2013-01-03 15:00:00+00:00,13381.66,13395.66,13376.42,13391.14
2013-01-03 16:00:00+00:00,13391.18,13413.73,13391.16,13408.19
2013-01-03 17:00:00+00:00,13408.16,13420.65,13402.45,13419.40
2013-01-03 18:00:00+00:00,13419.37,13430.13,13411.07,13427.51


In [7]:
sample2.to_json(orient='split', date_format="iso")

'{"columns":["Open","High","Low","Close"],"index":["2013-01-04T15:00:00.000Z","2013-01-04T16:00:00.000Z","2013-01-04T17:00:00.000Z"],"data":[[13407.46,13411.91,13377.22,13387.45],[13387.37,13420.86,13386.8,13408.33],[13408.2,13415.8,13403.52,13405.06]]}'

In [5]:
sample3

,Open,High,Low,Close
Date_Time,,,,
2013-01-02 15:00:00+00:00,13366.48,13374.82,13338.68,13345.45
2013-01-02 16:00:00+00:00,13345.60,13353.75,13321.64,13321.77
2013-01-02 17:00:00+00:00,13321.76,13339.25,13320.28,13326.66
2013-01-02 18:00:00+00:00,13326.64,13336.06,13322.51,13329.19
2013-01-02 19:00:00+00:00,13329.09,13339.82,13318.19,13331.80


In [24]:
def appendHDF(hdfFile, bucket, data, sample_unit='1H', update=False):
    # Get HDFStore
    hdfStore = pandas.HDFStore(hdfFile, 'a')
    append = True
    # TODO Sort incoming data
    
    if '/'+bucket in hdfStore.keys():
    
        # Get first,last row
        nrows = hdfStore.get_storer(bucket).nrows
        last = hdfStore.select(bucket,start=nrows-1,stop=nrows)
        
        # If this is entirely beyond the last element in the file... append
        # If not... update (incurring a full file re-write and performance hit), or throw exception                  
        if not data[data.index<=last.index[0]].empty:
            # Update table with overlapped data
            storedData = hdfStore.get(bucket)
            data = ppl.merge(data, storedData)
            append = False
            
            if not update:
                raise ValueError('Error: Entry already exists for data starting at index ' + str(data.index[0])) 
        else:                                      
            data = ppl.merge(last, data)        
     
    data = ppl.resample(data, sample_unit)
    if append:        
        print("Appending data...")
        hdfStore.append(bucket, data, format='table', append=True)
    else:
        print("Re-writing table data for update...")
        hdfStore.put(bucket, data, format='table')

    hdfStore.close()
    
    
def getHDF(hdfFile, bucket):
    
    # Get HDFStore
    hdfStore = pandas.HDFStore(hdfFile, 'r')
    data = hdfStore.get(bucket)   
    hdfStore.close()    
    return data



In [27]:
appendHDF("temp.hdf", "DOW", sample3, '1H', True)

Merging data...
Resampling to 1H periods
Re-writing table data for update...


In [18]:
getHDF("temp.hdf", "DOW")

,Open,High,Low,Close
Date_Time,,,,
2013-01-02 15:00:00+00:00,13366.48,13374.82,13338.68,13345.45
2013-01-02 16:00:00+00:00,13345.60,13353.75,13321.64,13321.77
2013-01-02 17:00:00+00:00,13321.76,13339.25,13320.28,13326.66
2013-01-02 18:00:00+00:00,13326.64,13336.06,13322.51,13329.19
2013-01-02 19:00:00+00:00,13329.09,13339.82,13318.19,13331.80
2013-01-02 20:00:00+00:00,13331.83,13395.20,13331.83,13395.20
2013-01-02 21:00:00+00:00,NaN,NaN,NaN,NaN
2013-01-02 22:00:00+00:00,NaN,NaN,NaN,NaN
2013-01-02 23:00:00+00:00,NaN,NaN,NaN,NaN
